In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [42]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [40]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [8]:
dependent=dataset[["Purchased"]]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid={'solver':['newton-cg','lbfgs','liblinear','saga'],
           'penalty':['l2']}
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)


C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    2.6s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    2.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    2.7s finished
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_

In [13]:
grid_predictions=grid.predict(x_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average="weighted")
print('the f1_macro value for best parameter{}:'.format(grid.best_params_),f1_macro)
print('the confusion matrix:\n',cm)
print('the report:\n',clf_report)


the f1_macro value for best parameter{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115367
the confusion matrix:
 [[74  5]
 [ 8 33]]
the report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9481321395492436

In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023514,0.017016,0.002671,0.003777,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.805615,0.739369,0.900383,0.815088,0.065961,1
1,0.010414,0.007364,0.000000,0.000000,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.805615,0.739369,0.900383,0.815088,0.065961,1
2,0.010417,0.007366,0.000000,0.000000,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.795673,0.739369,0.900383,0.811751,0.066604,4
3,0.005217,0.007379,0.000000,0.000000,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.805615,0.739369,0.900383,0.815088,0.065961,1


In [48]:
Gender_input=float(input("Gender:"))
Age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary:"))

 

Gender:1
Age:67
EstimatedSalary:5678548


In [49]:
Future_Prediction=grid.predict([[Gender_input,Age_input,EstimatedSalary_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
